In [1]:
import sqlite3
import pandas as pd
import numpy as np

### Connect to SQLite

In [2]:
con = sqlite3.connect("seoul_nomads.sqlite")
cur = con.cursor()

### Load file

In [3]:
df = pd.read_excel("/Users/fairlabs/Downloads/Stats.xlsx", sheet_name="Games")

In [4]:
def get_indexes(df):
    idxs = df[df['Unnamed: 1'].str.find("Game") == 0].index.tolist() + [df.shape[0] - 1]
    return idxs

In [5]:
def get_games(df, start, end):
    games = df.loc[start+1:end- 1,"Unnamed: 2":"Unnamed: 5"]
    games.dropna(axis=0, how='all', inplace=True)
    # games.drop(columns=['Unnamed: 2'], inplace=True)
    headers = ['game_number'] + games.iloc[0].tolist()[1:]
    games = pd.DataFrame(games.values[1:], columns=headers)
    return games

In [6]:
def get_game_part(df, start):
    if df.loc[start+1,"Unnamed: 12"] is not np.nan:
        game_part = df.loc[start+1,"Unnamed: 12"].split(' ')[1]
    else:
        game_part = 1
    return int(game_part)

In [7]:
def get_game_day(df, start):
    game_day = df.loc[start, 'Unnamed: 1'].split(' ')[1]
    game_day_reversed = game_day.split('.')[2] + '.' + game_day.split('.')[1] + '.' + game_day.split('.')[0]
    return game_day_reversed

In [8]:
def get_game_stats(games, game_day, game_part): 
    game_stats =[]
    for idx, row in games.iterrows():
        if row['Blue'] is np.nan:
            team_1 = (game_day, game_part, row['game_number'], 'Orange', 1 if row['Orange'] > row['Green'] else 0, 1 if row['Orange'] == row['Green'] else 0, 1 if row['Orange'] < row['Green'] else 0, row['Orange'] - row['Green'], row['Orange'], row['Green'])
            team_2 = (game_day, game_part, row['game_number'], 'Green', 1 if row['Orange'] < row['Green'] else 0, 1 if row['Orange'] == row['Green'] else 0, 1 if row['Orange'] > row['Green'] else 0, row['Green'] - row['Orange'], row['Green'], row['Orange'])
            game_stats.append(team_1)
            game_stats.append(team_2)
        if row['Orange'] is np.nan:
            team_1 = (game_day, game_part, row['game_number'], 'Blue', 1 if row['Blue'] > row['Green'] else 0, 1 if row['Blue'] == row['Green'] else 0, 1 if row['Blue'] < row['Green'] else 0, row['Blue'] - row['Green'], row['Blue'], row['Green'])
            team_2 = (game_day, game_part, row['game_number'], 'Green', 1 if row['Blue'] < row['Green'] else 0, 1 if row['Blue'] == row['Green'] else 0, 1 if row['Blue'] > row['Green'] else 0, row['Green'] - row['Blue'], row['Green'], row['Blue'])
            game_stats.append(team_1)
            game_stats.append(team_2)
        if row['Green'] is np.nan:
            team_1 = (game_day, game_part, row['game_number'], 'Blue', 1 if row['Blue'] > row['Orange'] else 0, 1 if row['Blue'] == row['Orange'] else 0, 1 if row['Blue'] < row['Orange'] else 0, row['Blue'] - row['Orange'], row['Blue'], row['Orange'])
            team_2 = (game_day, game_part, row['game_number'], 'Orange', 1 if row['Blue'] < row['Orange'] else 0, 1 if row['Blue'] == row['Orange'] else 0, 1 if row['Blue'] > row['Orange'] else 0, row['Orange'] - row['Blue'], row['Orange'], row['Blue'])
            game_stats.append(team_1)
            game_stats.append(team_2)
    return game_stats

In [9]:
def get_points(game_stats):
    points = pd.DataFrame(game_stats)
    points = points.groupby(3).sum()
    return points

In [10]:
def get_teams(df, start):
    teams = df.loc[start+2:start+7,"Unnamed: 7":"Unnamed: 9"]
    headers = teams.iloc[0].tolist()
    teams  = pd.DataFrame(teams.values[1:], columns=headers)
    return teams

In [11]:
def get_players(teams, game_day, game_part):
    players = []
    for team_color in teams.columns:
        for player in teams[team_color]:
            if player is not np.nan:
                player_info = (game_day, game_part, player[:-4] if player.endswith('(c)') else player, team_color, 1 if player.endswith('(c)') else 0)
                players.append(player_info)
    return players

In [12]:
idxs = get_indexes(df)

In [13]:
def get_overall_games(df, start, end):
    idxs = get_indexes(df)
    game_dfs = []
    for i in range(start, end):
        games = get_games(df, idxs[i], idxs[i+1])
        game_part = get_game_part(df, idxs[i])
        game_day = get_game_day(df, idxs[i])
        game_stats = get_game_stats(games, game_day, game_part)
        game_dfs.extend(game_stats)
    return game_dfs

In [14]:
ovrl_gms = get_overall_games(df, 0, len(idxs)-1)

In [15]:
def get_overall_players(df, start, end):
    idxs = get_indexes(df)
    player_dfs = []
    for i in range(start, end):
        teams = get_teams(df, idxs[i])
        game_part = get_game_part(df, idxs[i])
        game_day = get_game_day(df, idxs[i])
        players = get_players(teams, game_day, game_part)
        player_dfs.extend(players)
    return player_dfs

In [16]:
ovrl_pls = get_overall_players(df, 0, len(idxs)-1)

In [17]:
len(ovrl_pls)

542

### Drop Tables

In [18]:
cur.execute("""DROP TABLE players""")

In [19]:
cur.execute("""DROP TABLE games""")

### Create Tables

In [20]:
cur.execute("""CREATE TABLE 
                players(
                    date varchar, 
                    game_part int, 
                    name varchar, 
                    team varchar, 
                    captain integer)""")

In [21]:
cur.execute("""CREATE TABLE 
                games(
                    date varchar, 
                    game_part int, 
                    game_number int, 
                    team varchar, 
                    win integer, 
                    draw integer, 
                    lose integer, 
                    goal_difference integer, 
                    goals_scored integer, 
                    goals_conceded integer)""")

### Insert to DB

In [37]:
q = "SELECT count(p.team), p.name FROM players as p JOIN games as g ON p.team = g.team and p.date = g.date and p.game_part=g.game_part WHERE p.name <> 'Nurseit' and p.name <> 'Nauryzbay' GROUP BY p.name"
res = cur.execute(q)

In [44]:
q = "SELECT * from games as g join players as p on p.team = g.team and p.date = g.date and p.game_part=g.game_part where name = 'Argo' ORDER BY g.date desc, g.game_part desc, g.game_number desc"
res = cur.execute(q)

In [45]:
res.fetchall()

[('23.01.14',
  1,
  15,
  'Blue',
  0,
  0,
  1,
  -2,
  0,
  2,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14',
  1,
  13,
  'Blue',
  0,
  1,
  0,
  0,
  1,
  1,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14',
  1,
  12,
  'Blue',
  1,
  0,
  0,
  2,
  2,
  0,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14',
  1,
  11,
  'Blue',
  1,
  0,
  0,
  1,
  2,
  1,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14',
  1,
  9,
  'Blue',
  0,
  0,
  1,
  -1,
  0,
  1,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14',
  1,
  8,
  'Blue',
  0,
  1,
  0,
  0,
  0,
  0,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14',
  1,
  6,
  'Blue',
  0,
  0,
  1,
  -1,
  0,
  1,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14',
  1,
  5,
  'Blue',
  0,
  1,
  0,
  0,
  0,
  0,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14',
  1,
  3,
  'Blue',
  0,
  0,
  1,
  -2,
  0,
  2,
  '23.01.14',
  1,
  'Argo',
  'Blue',
  1),
 ('23.01.14

In [25]:
def insert_games(games):
    for game in games:
        q = f"""INSERT INTO games(date, game_part, game_number, team, win, draw, lose, goal_difference, goals_scored, goals_conceded) VALUES('{(game[0])}',{game[1]},{game[2]},'{game[3]}',{game[4]},{game[5]},{game[6]},{game[7]},{game[8]},{game[9]})"""
        cur.execute(q)

In [26]:
insert_games(ovrl_gms)

In [27]:
def insert_players(players):
    for player in players:
        q = f"""INSERT INTO players(date, game_part, name, team, captain) VALUES('{(player[0])}', {player[1]},'{player[2]}','{player[3]}',{player[4]})"""
        cur.execute(q)

In [28]:
insert_players(ovrl_pls)

### Delete from DB

cur.execute("DELETE FROM games")
cur.fetchall()

In [29]:
def get_player_names():
    q = "SELECT p.name FROM players as p JOIN games as g ON p.team = g.team and p.date = g.date and p.game_part=g.game_part WHERE p.name <> 'Nurseit' and p.name <> 'Nauryzbay' GROUP BY p.name having count(*) > 50"
    cur.execute(q)
    res = cur.fetchall()
    player_names = list(map(lambda x: x[0], res))
    return player_names

In [30]:
player_names = get_player_names()

In [31]:
player_names

['Abdirakhim',
 'Aidyn',
 'Almas A.',
 'Almas S.',
 'Argo',
 'Azamatbek',
 'Baurzhan',
 'Bayram',
 'Beksultan',
 'Bekzhan',
 'Bolat',
 'Chingiz',
 'Daniyar',
 'Dauren',
 'Dulan',
 'Jahongir',
 'Kanat',
 'Kanye',
 'Muhammadjon',
 'Nazarbek',
 'Sanzhar',
 'Silap',
 'Temirlan',
 'Tynychbek',
 'Yerzhan',
 'Yessen',
 'Zakirbek']

In [32]:
def get_overall_stats(player_names):
    overall_stats = []
    for player_name in player_names:
        q = f"""SELECT 
                    name as Name, 
                    1.0*sum(win*3+draw)/count(*) as 'Pts/G',
                    1.0*sum(goal_difference)/count(*) as 'GD/G', 
                    1.0*sum(goals_scored)/count(*) as 'GF/G', 
                    1.0*sum(goals_conceded)/count(*) as 'GA/G', 
                    count(*) as Games
                FROM 
                    (SELECT 
                        * 
                    FROM 
                        players as p 
                    JOIN 
                        games as g 

                    ON p.team = g.team 
                        and 
                        p.date = g.date 
                        and 
                        p.game_part=g.game_part 
                    WHERE 
                        p.name = '{player_name}' 
                    ORDER BY 
                        g.date desc, g.game_part desc, g.game_number desc limit 50)"""
        cur.execute(q)
        res = cur.fetchall()
        overall_stats.extend(res)
    return overall_stats

In [49]:
res = get_overall_stats(player_names)

### Get Overall Stats

In [50]:
stats = pd.DataFrame(res)

In [51]:
stats = stats.sort_values(by=[1, 2], ascending=False).reset_index(drop=True)

In [52]:
stats

,0,1,2,3,4,5
0,Baurzhan,1.66,0.24,0.96,0.72,50
1,Temirlan,1.66,0.22,0.96,0.74,50
2,Zakirbek,1.66,0.22,0.88,0.66,50
3,Abdirakhim,1.52,0.12,0.78,0.66,50
4,Sanzhar,1.50,0.20,0.84,0.64,50
5,Yerzhan,1.46,0.08,0.88,0.80,50
6,Beksultan,1.42,0.18,0.92,0.74,50
7,Almas A.,1.42,0.10,0.98,0.88,50
8,Kanye,1.38,0.16,0.86,0.70,50
9,Jahongir,1.36,0.04,0.86,0.82,50


In [ ]:
stats.to_csv('stats.csv', index=None)